<a href="https://colab.research.google.com/github/lfernandof/lfp_signals_analysis/blob/main/state-scored-spikes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import json
import scipy

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [118]:
def scored_state_probing(rat_id,day_id,session_data,state_type='NREMstate'):
  '''
  Find all intervals of a given type in a session and construct DataFrames with all events (spikes) comprising each period, alongside other relevant columns.

  Parameters
  ----------
  rat_id: int
          The chosen animal
  day_id: int
          The day in which the session happened
  state_type: string [default: "NREMstate"]
          The type of period which to probe. Options are: "NREMstate", "WAKEstate", "REMstate" and "events_pathologicalstate"

  Returns
  ----------
  session_staging_df: pandas DataFrame
          a pandas DataFrame with the following columns:
            - timestamp [float]: instant at which a given spike happened (in s)
            - region [string]: either 'PFC' or 'STR'
            - day [int]: either '1' or '2'
            - group [int]: only '2' for now, for we only have pre-trial spiking data from animals in group 2
            - rat [int]: each animal's ID
            - stage_type [string]: either {'WAKE','nREM','REM' or 'pathological'}
            - period_id [int]: period index for a given stage (e.g.: {stage_type = 'NREM',period_id = 0} is the first NREM period for that neuron in that session)
            - duration [float]: the length of a given period in s (e.g. {stage_type = 'NREM',period_id = 0, duration = 15} indicates that the first NREM period for that neuron in that session lasts 15 s)

  '''
  print(f'Beginning the stage segmentation of {state_type.removesuffix("state")} states from session:\n(Rat: {rat_id}, day: {day_id})')

  #Query for the desired session and find the first and last timestamp, corresponding to the beginning and end of the training protocol during this session
  training_begin = session_data.query(f'rat == {rat_id} and day == {day_id}').timestamp.min()
  training_end = session_data.query(f'rat == {rat_id} and day == {day_id}').timestamp.max()

  #Load the timestamps for each event (e.g. sleep, pathological states) as a list

  #We set up these conditionals just for rat 4, whose folders are differently named from the pattern
  if rat_id == 4 and day_id == 1:
    day_tag = '2017-05-06_09-58-35'
  elif rat_id == 4 and day_id == 2:
    day_tag = '2017-05-07_13-57-27'
  else:
    day_tag = f'Day{day_id}'

  state_scores_path = f'/content/drive/MyDrive/MSc/Rat{rat_id}/{day_tag}'

  #Load the state scoring timestamps proper
  data_states = sio.loadmat(state_scores_path+f'/{day_tag}.SleepState.states.mat')

  #Not all sessions have pathological states
  if state_type == 'events_pathologicalstate':
    try:
      event_timestamps = data_states['SleepState']['ints'][0][0][state_type][0][0]
    except:
      raise('There are no pathological (microseizure) states in this session.')
  #But all others are guaranteed
  else:
    event_timestamps = data_states['SleepState']['ints'][0][0][state_type][0][0]

  # Load the spike corresponding to this session
  data_path = f'drive/MyDrive/MSc/Rat{rat_id}/{day_tag}/'
  spikefile_tag = 'openephys'
  #exceptions to overwrite names because these folders/files are wrongly named and I do not have permission to change it
  if rat_id == 4 and day_id == 1:
    spikefile_tag = '2017-05-06_09-58-35'
  spikes_test = sio.loadmat(data_path+spikefile_tag+'.spikes.cellinfo.mat')

  #UID (unit id)
  uid = np.squeeze(spikes_test['spikes'][0][0]['UID'])

  #times (spike times)
  times = np.squeeze(spikes_test['spikes'][0][0]['times'])

  #the region each unit belongs to
  region = np.squeeze(spikes_test['spikes'][0][0]['region'])

  #shankID, cluID, rawWaveform, maxWaveformCh are also in this .mat file, but I will not be using any of these


  def create_or_append(dataframe,current_data):
    if dataframe is not None:
        return pd.concat([dataframe,pd.DataFrame(current_data)])
    else:
        return pd.DataFrame(current_data)

  session_df = None #empty variable; upon first being called a df it will be rewritten by a df; upon next calls in the loop it will be concatenated with newer inputs

  #iterate over neurons
  for unit_id in np.squeeze(uid)-1: #uid is 1-indexed as per matlab, whereas python used 0-indexing
    current_spike_train = np.squeeze(times[unit_id])
    current_unit_region = np.squeeze(region)[unit_id][0]

    #for each neuron find which of its spikes fall within each period (stage) of a given type (e.g. sleep) and add each spike as a row in a pandas DataFrame
    for period_id,period_limits in enumerate(event_timestamps):
      period_timestamps = current_spike_train[np.where((current_spike_train > period_limits[0]) * (current_spike_train < period_limits[1]))]
      period_df = pd.DataFrame.from_dict({'timestamp':period_timestamps,'region':current_unit_region,'day':day_id,'group':group_id,
                                          'neuron':unit_id,'stage':state_type,'stage_id':period_id,'stage_durtion':np.diff(period_limits)[0]})
      session_df = create_or_append(session_df,period_df)

  print('Done')
  return session_df

In [119]:
session_data = pd.read_csv('/content/drive/MyDrive/backup-mac/ufabc/entire-backup/lfp/all_channels//data/session_data-spikes.csv').copy()

rat_id = 6
day_id = 1
test_df = scored_state_probing(rat_id,day_id,session_data,state_type='NREMstate')

Beginning the stage segmentation of NREM states from session:
(Rat: 6, day: 1)
Done


There is one problem left to solve: session _rat 4, day1_ does not have a 'region' identifier in the _.mat_ file. I will have to infer each region from _shankID_, cross-referencing other sessions' files. I will do that tomorrow.